In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
from model_mnist import LeNet5
from visualize import *
import dset_mnist as dset
import foolbox
sys.path.append('../trim')
from transforms_torch import transform_bandpass, tensor_t_augment, batch_fftshift2d, batch_ifftshift2d
from trim import *
from util import *
from attributions import *
from captum.attr import *
import warnings
warnings.filterwarnings("ignore")
# disentangled vae
sys.path.append('../disentangling-vae')
from collections import defaultdict
import vae_trim, vae_trim_viz
from disvae.utils.modelIO import save_model, load_model, load_metadata
from disvae.models.losses import get_loss_f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'vae_trim'

In [5]:
s

NameError: name 's' is not defined

### BTCVAE-Attr-Pen

In [2]:
from disvae import init_specific_model
from utils.datasets import get_img_size

In [3]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 0 # total correlation reg
args.attr_lamb = 1 # change of attribute wrt other attributes
name = args.loss + "_B_" + str(args.btcvae_B) + "_attr_" + str(args.attr_lamb)
args.name = name

In [4]:
# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)

# initialize model
args.img_size = get_img_size(args.dataset)
model = init_specific_model(args.model_type, args.img_size, args.latent_dim).to(device)

# Train
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# loss
L1Loss = torch.nn.L1Loss()
# L1Loss = torch.nn.MSELoss()
loss_f = get_loss_f(args.loss,
                    n_data=len(train_loader.dataset),
                    device=device,
                    **vars(args))

# saliency map
saliency = InputXGradient(DecoderEncoder(model))

Train model

In [5]:
model.train()
for epoch in range(5):
    epoch_loss = 0
    epoch_reg = 0
    # one epoch
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)    
        batch_size, channel, height, width = inputs.size()
        recon_batch, latent_dist, latent_sample = model(inputs)

        loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                           None, latent_sample=latent_sample)        
        # penalize change in one attribute wrt the other attributes
        reg = 0
        if args.attr_lamb > 0:
            s = latent_dist[0]
            for i in range(model.latent_dim):
                col_idx = np.arange(model.latent_dim)!=i
#                 attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
                attributions = saliency.attribute(s, target=i)[:,col_idx]
                reg += args.attr_lamb * L1Loss(attributions, torch.zeros_like(attributions))
        loss += reg
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        epoch_loss += loss.item()
        epoch_reg += reg.item()      
        print('\rTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(labels), len(train_loader.dataset),
                   100. * batch_idx / len(train_loader), loss.data.item()), end='')        
    mean_epoch_loss = epoch_loss / len(train_loader)
    mean_epoch_reg = epoch_reg / len(train_loader)
    print('\nAverage loss: {} - Average reg: {}'.format(mean_epoch_loss, mean_epoch_reg))

Train Epoch: 0 [29984/60000 (100%)]	Loss: 182.427353
Average loss: 227.25967573153693 - Average reg: 0.9099381917289325
Train Epoch: 1 [29984/60000 (100%)]	Loss: 175.624710
Average loss: 184.14279698613865 - Average reg: 0.8886128656391395
Train Epoch: 2 [29984/60000 (100%)]	Loss: 178.429245
Average loss: 179.41855105662395 - Average reg: 0.8526769884740875
Train Epoch: 3 [29984/60000 (100%)]	Loss: 172.840973
Average loss: 176.85795482643633 - Average reg: 0.8311308870818823
Train Epoch: 4 [29984/60000 (100%)]	Loss: 173.116241
Average loss: 175.0110388879837 - Average reg: 0.8156537078718132


In [33]:
model.eval()
epoch_loss = 0
epoch_reg = 0
for batch_idx, (inputs, labels) in enumerate(train_loader):     
    inputs = inputs.to(device)
    labels = labels.to(device)    
    batch_size, channel, height, width = inputs.size()
    recon_batch, latent_dist, latent_sample = model(inputs)    
    
    loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                       None, latent_sample=latent_sample)     
    # penalize change in one attribute wrt the other attributes
    reg = 0
    s = deepcopy(latent_dist[0].detach())
    for i in range(model.latent_dim):
        col_idx = np.arange(model.latent_dim)!=i
#         attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
        attributions = saliency.attribute(s, target=i)[:,col_idx]
        reg += L1Loss(attributions, torch.zeros_like(attributions))
    epoch_loss += loss.item()
    epoch_reg += reg.item()  
print(epoch_loss/len(train_loader), epoch_reg/len(train_loader))

173.06489894842542 0.7909434062204381


In [110]:
inputs, labels = iter(train_loader).next()
inputs = inputs.to(device)
labels = labels.to(device)    
batch_size, channel, height, width = inputs.size()
recon_batch, latent_dist, latent_sample = model(inputs) 

In [119]:
reg = 0
s = latent_dist[0]
m = DecoderEncoder(model)
optimizer = optim.Adam(m.parameters(), lr=args.lr)
optimizer.zero_grad()

s = s.requires_grad_(True)
s_ = m(s)
for i in range(model.latent_dim):
    musk = torch.ones_like(s_)
    musk[:,i] = 0
    gradients = torch.autograd.grad(s_[:,i], s, grad_outputs=torch.ones_like(s_), retain_graph=True, create_graph=True, only_inputs=True)

In [123]:
torch.ones_like(s_)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],


In [116]:
s_.shape

torch.Size([64, 10])

In [ ]:
    for i in range(model.latent_dim):
        col_idx = np.arange(model.latent_dim)!=i
#         attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
        attributions = saliency.attribute(s, target=i)[:,col_idx]
        reg += L1Loss(attributions, torch.zeros_like(attributions))

In [107]:
m.encoder.conv1.weight.grad.sum()

tensor(0., device='cuda:0')

In [109]:
gradients[0].shape

torch.Size([64, 10])

In [76]:
gradients = autograd.grad(output, input_, grad_outputs=musk,
                          retain_graph=True, create_graph=True,
                          allow_unused=True)[0]  # get tensor from tuple
gradients = gradients.view(-1, 1)
gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()

NameError: name 'autograd' is not defined

In [ ]:
torch.autograd.gradcheck

In [ ]:
def compute_gradient_penalty(self, network, input):
    # ref: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py
    '''

    :param input: state[index]
    :param network: actor or critic
    :return: gradient penalty
    '''
    input_ = torch.tensor(input).requires_grad_(True)
    output = network(input_)
    musk = torch.ones_like(output)
    gradients = grad(output, input_, grad_outputs=musk,
                     retain_graph=True, create_graph=True,
                     allow_unused=True)[0]  # get tensor from tuple
    gradients = gradients.view(-1, 1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
gradients = autograd.grad(output, input_, grad_outputs=musk,
                          retain_graph=True, create_graph=True,
                          allow_unused=True)[0]  # get tensor from tuple
gradients = gradients.view(-1, 1)
gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()

In [50]:
reg = 0
s = deepcopy(latent_dist[0].detach())
m = DecoderEncoder(model)
optimizer = optim.Adam(m.parameters(), lr=args.lr)
optimizer.zero_grad()




    gradients = grad(output, input_, grad_outputs=musk,
                     retain_graph=True, create_graph=True,
                     allow_unused=True)[0]  # get tensor from tuple
    gradients = gradients.view(-1, 1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [44]:
reg = 0
s = latent_dist[0]
m = DecoderEncoder(model)
attr = InputXGradient(m)
optimizer = optim.Adam(m.parameters(), lr=args.lr)
optimizer.zero_grad()

col_idx = np.arange(model.latent_dim)!=0
attributions = torch.div(attr.attribute(s, target=0),s)[:,col_idx]
reg = L1Loss(attributions, torch.zeros_like(attributions))

In [ ]:
def compute_gradient_penalty(self, network, input):
    # ref: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py
    '''

    :param input: state[index]
    :param network: actor or critic
    :return: gradient penalty
    '''
    input_ = torch.tensor(input).requires_grad_(True)
    output = network(input_)
    musk = torch.ones_like(output)
    gradients = grad(output, input_, grad_outputs=musk,
                     retain_graph=True, create_graph=True,
                     allow_unused=True)[0]  # get tensor from tuple
    gradients = gradients.view(-1, 1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [47]:
reg.backward()

In [48]:
m.encoder.conv1.weight.grad.sum()

tensor(6.9506e-09, device='cuda:0')